<a href="https://colab.research.google.com/github/drstannwoji2019/ML_Projects/blob/main/DL_REM_FDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, Dropout, LayerNormalization, MultiHeadAttention, Input, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Load the dataset
file_path = "FDI_Rem_Analysis_2004_2022.csv"  # Ensure this file is in your working directory
df = pd.read_csv('/FDI_Rem_Analysis_2004_2022.csv')

# Convert 'Year' to datetime format and set as index
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df.set_index('Year', inplace=True)

# Selecting target variables (FDI and Remittances)
targets = ['FDI_Total', 'Rem_Total']

# Normalize the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[targets])

# Split data into training (2004-2018) and testing (2019-2022)
train_size = int(len(df) * 0.8)
train_data = df_scaled[:train_size]
test_data = df_scaled[train_size:]

# Function to create sequences for LSTM, GRU, RNN, and Transformer
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Define sequence length
SEQ_LENGTH = 3

# Prepare sequences
X_train, y_train = create_sequences(train_data, SEQ_LENGTH)
X_test, y_test = create_sequences(test_data, SEQ_LENGTH)

# Reshape inputs for LSTM/GRU/RNN (samples, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], len(targets)))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], len(targets)))

# Function to build and train models
def build_and_train_model(model_name, model_type):
    if model_type == "Transformer":
        # Define input layer explicitly
        input_layer = Input(shape=(SEQ_LENGTH, len(targets)))

        # Apply MultiHeadAttention to the input layer
        attention_output = MultiHeadAttention(num_heads=2, key_dim=2)(input_layer, input_layer, input_layer)  # Pass input_layer as query, key, and value

        # Continue with other layers
        x = LayerNormalization()(attention_output)
        # Adjust the output dimension of the Dense layer to match the target shape
        x = Flatten()(x)
        x = Dense(len(targets), activation='linear')(x)  # Added Dense layer

        # Create the model
        model = tf.keras.Model(inputs=input_layer, outputs=x)
    else:
        model = Sequential()
        if model_type == "LSTM":
            model.add(LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, len(targets))))
            model.add(LSTM(50))
        elif model_type == "GRU":
            model.add(GRU(50, return_sequences=True, input_shape=(SEQ_LENGTH, len(targets))))
            model.add(GRU(50))
        elif model_type == "RNN":
            model.add(SimpleRNN(50, return_sequences=True, input_shape=(SEQ_LENGTH, len(targets))))
            model.add(SimpleRNN(50))

        model.add(Dense(25, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(len(targets), activation='linear'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Train model
    model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=1, validation_data=(X_test, y_test))

    # Predictions
    predictions = model.predict(X_test)

    # Inverse scale predictions and actual values
    predictions_rescaled = scaler.inverse_transform(predictions)
    y_test_rescaled = scaler.inverse_transform(y_test)

    # Calculate performance metrics
    results = {
        "FDI_MAE": mean_absolute_error(y_test_rescaled[:, 0], predictions_rescaled[:, 0]),
        "FDI_MSE": mean_squared_error(y_test_rescaled[:, 0], predictions_rescaled[:, 0]),
        "FDI_R2": r2_score(y_test_rescaled[:, 0], predictions_rescaled[:, 0]),
        "Rem_MAE": mean_absolute_error(y_test_rescaled[:, 1], predictions_rescaled[:, 1]),
        "Rem_MSE": mean_squared_error(y_test_rescaled[:, 1], predictions_rescaled[:, 1]),
        "Rem_R2": r2_score(y_test_rescaled[:, 1], predictions_rescaled[:, 1]),
    }

    return model_name, results

    # Predictions
    predictions = model.predict(X_test)

    # Inverse scale predictions and actual values
    predictions_rescaled = scaler.inverse_transform(predictions)
    y_test_rescaled = scaler.inverse_transform(y_test)

    # Calculate performance metrics
    results = {
        "FDI_MAE": mean_absolute_error(y_test_rescaled[:, 0], predictions_rescaled[:, 0]),
        "FDI_MSE": mean_squared_error(y_test_rescaled[:, 0], predictions_rescaled[:, 0]),
        "FDI_R2": r2_score(y_test_rescaled[:, 0], predictions_rescaled[:, 0]),
        "Rem_MAE": mean_absolute_error(y_test_rescaled[:, 1], predictions_rescaled[:, 1]),
        "Rem_MSE": mean_squared_error(y_test_rescaled[:, 1], predictions_rescaled[:, 1]),
        "Rem_R2": r2_score(y_test_rescaled[:, 1], predictions_rescaled[:, 1]),
    }

    return model_name, results

# Train models and store results
deep_learning_results = {}
for model_name, model_type in zip(["LSTM", "GRU", "RNN", "Transformer"], ["LSTM", "GRU", "RNN", "Transformer"]):
    name, result = build_and_train_model(model_name, model_type)
    deep_learning_results[name] = result

# Convert results to DataFrame and display
deep_learning_results_df = pd.DataFrame(deep_learning_results).T

# Print results
print("\nDeep Learning Model Performance:")
print(deep_learning_results_df)


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 757ms/step - loss: 0.4985 - val_loss: 0.3980
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - loss: 0.4753 - val_loss: 0.3763
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.4282 - val_loss: 0.3536
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.3931 - val_loss: 0.3304
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.3415 - val_loss: 0.3066
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.3163 - val_loss: 0.2830
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.2873 - val_loss: 0.2598
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.2277 - val_loss: 0.2390
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.2118 - val_loss: 0.2192
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.1671 - val_loss: 0.2027
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.1303 - val_loss: 0.1933
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0869 - val_loss: 0.1903
Epoch 13

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 748ms/step - loss: 0.4530 - val_loss: 0.3506
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.4059 - val_loss: 0.3179
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.3188 - val_loss: 0.2807
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.2839 - val_loss: 0.2464
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.1851 - val_loss: 0.2180
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1337 - val_loss: 0.1968
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0900 - val_loss: 0.1832
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.1034 - val_loss: 0.1797
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0591 - val_loss: 0.1792
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 0.0819 - val_loss: 0.1729
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0524 - val_loss: 0.1607
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0795 - val_loss: 0.1408
Epoch 13/5

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 693ms/step - loss: 0.1126 - val_loss: 0.0523
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.1293 - val_loss: 0.0459
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.0734 - val_loss: 0.1001
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.1438 - val_loss: 0.0792
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0572 - val_loss: 0.0474
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0674 - val_loss: 0.0329
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0579 - val_loss: 0.0342
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0409 - val_loss: 0.0380
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0598 - val_loss: 0.0439
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.1074 - val_loss: 0.0503
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0470 - val_loss: 0.0505
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0358 - val_loss: 0.0507
Epoch 13/

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 286ms/step - loss: 1.5619 - val_loss: 1.7997
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 1.5963 - val_loss: 1.7699
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.6182 - val_loss: 1.7389
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.5169 - val_loss: 1.7068
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.5420 - val_loss: 1.6752
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.4086 - val_loss: 1.6415
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 1.4277 - val_loss: 1.6082
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 1.2707 - val_loss: 1.5754
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.2264 - val_loss: 1.5431
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 1.1012 - val_loss: 1.5112
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 1.0432 - val_loss: 1.4805
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.9228 - val_loss: 1.4492
Epoch 13/5

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
